In [12]:
"""
Script para la preparación final de datos antes del modelado.

Incluye:
1. Análisis de Componentes Principales (PCA) para exploración.
2. División de datos en conjuntos de entrenamiento y prueba.
3. Balanceo del conjunto de entrenamiento usando la técnica SMOTE.
4. Guardado de los conjuntos de datos procesados para el siguiente paso.
"""

# 1. IMPORTACIÓN DE LIBRERÍAS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from collections import Counter
from pathlib import Path

In [13]:
# 2. CONFIGURACIÓN Y CARGA DE DATOS
# Definir rutas de entrada y salida
BASE_PATH = Path(r"B:\Documentos\Universidad\Universidad 2025\Primavera\Machine Learning\segundo intento modelo ML\Codigos")
INPUT_FILE = BASE_PATH / "dataset_unificado_limpio.csv"
OUTPUT_PATH = BASE_PATH # Guardar en la misma carpeta

# Cargar el dataset pre-procesado
df = pd.read_csv(INPUT_FILE, parse_dates=['Datetime'], index_col='Datetime')

# Separar características (X) y variable objetivo (y)
# Se seleccionan las variables predictoras.
features = ['CO', 'Humedad', 'NO', 'NO2', 'NOx', 'O3', 'PM10', 'SO2', 'Temp', 'Viento_Cos', 'Viento_Sin']
X = df[features]
y = df['Calidad_Aire']

print("Datos cargados correctamente.")
print(f"Dimensiones de X: {X.shape}")
print(f"Dimensiones de y: {y.shape}")


Datos cargados correctamente.
Dimensiones de X: (49703, 11)
Dimensiones de y: (49703,)


In [14]:
# 3. ANÁLISIS DE COMPONENTES PRINCIPALES (PCA) EN GASES
print("\nIniciando análisis PCA sobre gases nitrogenados...")
# Seleccionamos un subconjunto de variables altamente correlacionadas para el análisis.
gases_features = ['NO', 'NO2', 'NOx']
X_gases = df[gases_features]

# Es fundamental estandarizar los datos antes de aplicar PCA.
scaler_gases = StandardScaler()
X_gases_scaled = scaler_gases.fit_transform(X_gases)

# Aplicar PCA para reducir de 3 a 2 dimensiones.
pca = PCA(n_components=2)
principal_components = pca.fit_transform(X_gases_scaled)

# Evaluar la cantidad de información (varianza) que retiene cada componente.
varianza = pca.explained_variance_ratio_
print(f"Varianza explicada por Componente 1: {varianza[0]*100:.2f}%")
print(f"Varianza explicada por Componente 2: {varianza[1]*100:.2f}%")
print(f"Información total conservada: {sum(varianza)*100:.2f}%")

# Visualización del resultado de PCA
plt.figure(figsize=(10, 6))
plt.scatter(principal_components[:, 0], principal_components[:, 1], c=df['NOx'], cmap='viridis', alpha=0.2)
plt.title('Proyección PCA de Gases Nitrogenados')
plt.xlabel('Componente Principal 1 (Magnitud general de contaminación)')
plt.ylabel('Componente Principal 2 (Variación entre gases)')
plt.colorbar(label='Concentración de NOx')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(OUTPUT_PATH / "analisis_pca_gases.png", dpi=200)
plt.close()
print("Gráfico PCA guardado.")



Iniciando análisis PCA sobre gases nitrogenados...
Varianza explicada por Componente 1: 86.03%
Varianza explicada por Componente 2: 13.97%
Información total conservada: 100.00%
Gráfico PCA guardado.


In [15]:
# 4. DIVISIÓN EN CONJUNTOS DE ENTRENAMIENTO Y PRUEBA
print("\nDividiendo datos en entrenamiento (70%) y prueba (30%)...")
# Se usa 'stratify=y' para mantener la proporción de clases en ambos conjuntos.
# 'random_state' asegura que la división sea siempre la misma.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
print("División completada.")



Dividiendo datos en entrenamiento (70%) y prueba (30%)...
División completada.


In [16]:
# 5. BALANCEO DE CLASES CON SMOTE (SOLO EN DATOS DE ENTRENAMIENTO)
print("\nAplicando SMOTE para balancear el conjunto de entrenamiento...")
counter_before = Counter(y_train)
print("Conteo de clases ANTES de SMOTE:", dict(sorted(counter_before.items())))

# Inicializar y aplicar SMOTE.
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

counter_after = Counter(y_train_smote)
print("Conteo de clases DESPUÉS de SMOTE:", dict(sorted(counter_after.items())))



Aplicando SMOTE para balancear el conjunto de entrenamiento...
Conteo de clases ANTES de SMOTE: {0: 19384, 1: 8169, 2: 2414, 3: 2841, 4: 1984}
Conteo de clases DESPUÉS de SMOTE: {0: 19384, 1: 19384, 2: 19384, 3: 19384, 4: 19384}


In [17]:
# 6. VISUALIZACIÓN DEL EFECTO DE SMOTE
# Preparar datos para el gráfico comparativo
labels_sorted = sorted(counter_before.keys())
v_before_sorted = [counter_before[k] for k in labels_sorted]
v_after_sorted = [counter_after[k] for k in labels_sorted]
nombres_clases = ['0: Muy Bueno', '1: Bueno', '2: Regular', '3: Malo', '4: Muy Malo']

# Crear gráfico
fig, ax = plt.subplots(figsize=(12, 7))
x_pos = np.arange(len(labels_sorted))
width = 0.35

rects1 = ax.bar(x_pos - width/2, v_before_sorted, width, label='Original (Desbalanceado)', color='skyblue')
rects2 = ax.bar(x_pos + width/2, v_after_sorted, width, label='Con SMOTE (Balanceado)', color='salmon')

ax.set_ylabel('Número de Muestras')
ax.set_title('Comparación de Distribución de Clases Antes y Después de SMOTE')
ax.set_xticks(x_pos)
ax.set_xticklabels(nombres_clases)
ax.legend()
ax.grid(axis='y', linestyle='--', alpha=0.7)
ax.bar_label(rects1, padding=3, fmt='%d')
ax.bar_label(rects2, padding=3, fmt='%d')

plt.tight_layout()
plt.savefig(OUTPUT_PATH / "balanceo_smote_comparativo.png", dpi=200)
plt.close()
print("Gráfico comparativo de SMOTE guardado.")


Gráfico comparativo de SMOTE guardado.


In [18]:
# 7. GUARDADO DE LOS DATASETS FINALES
print("\nGuardando los conjuntos de datos procesados en archivos CSV...")
# Guardamos los sets listos para ser usados en el entrenamiento del modelo.
X_train_smote.to_csv(OUTPUT_PATH / "X_train_balanced.csv", index=False)
y_train_smote.to_csv(OUTPUT_PATH / "y_train_balanced.csv", index=False)
X_test.to_csv(OUTPUT_PATH / "X_test.csv", index=False)
y_test.to_csv(OUTPUT_PATH / "y_test.csv", index=False)

print("\n[PROCESO COMPLETADO]")
print(f"Archivos guardados en: '{OUTPUT_PATH}'")


Guardando los conjuntos de datos procesados en archivos CSV...

[PROCESO COMPLETADO]
Archivos guardados en: 'B:\Documentos\Universidad\Universidad 2025\Primavera\Machine Learning\segundo intento modelo ML\Codigos'
